In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
movies_metadata = pd.read_csv('/content/movies_metadata.csv', low_memory=False)
keywords = pd.read_csv('/content/keywords.csv')
credits = pd.read_csv('/content/credits.csv')
links = pd.read_csv('/content/links.csv')
links_small = pd.read_csv('/content/links.csv')
ratings_small = pd.read_csv('/content/movies_metadata.csv')
movies_metadata['release_date'] = pd.to_datetime(movies_metadata['release_date'], errors='coerce')
movies_metadata['genres'] = movies_metadata['genres'].fillna('[]').apply(eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
sns.histplot(ratings_small['rating'], bins=10, kde=True)
plt.title('Distribution of Ratings')
plt.show()
tfidf = TfidfVectorizer(stop_words='english')
movies_metadata['overview'] = movies_metadata['overview'].fillna('')
tfidf_matrix = tfidf.fit_transform(movies_metadata['overview'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
def get_recommendations(title, cosine_sim=cosine_sim):
    indices = pd.Series(movies_metadata.index, index=movies_metadata['title']).drop_duplicates()
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    return movies_metadata['title'].iloc[movie_indices]
sparse_user_ratings_matrix = csr_matrix(user_ratings_matrix)
U, sigma, Vt = svds(sparse_user_ratings_matrix, k=50)
sigma = np.diag(sigma)
predicted_ratings = np.dot(np.dot(U, sigma), Vt)

def recommend_movies(user_id, num_recommendations=5):
    user_row_number = user_id - 1
    sorted_user_predictions = predicted_ratings[user_row_number].argsort()[::-1]
    movie_indices = sorted_user_predictions[:num_recommendations]
    return movies_metadata['title'].iloc[movie_indices]

print(recommend_movies(1))
predicted_ratings_df = pd.DataFrame(predicted_ratings, columns=user_ratings_matrix.columns)
true_ratings = ratings_small.set_index(['userId', 'movieId'])
predicted_ratings_series = predicted_ratings_df.stack().rename_axis(['userId', 'movieId'])
plt.plot(range(1, 6), [get_recommendations('The Godfather')] * 5, label='Content-Based')
plt.plot(range(1, 6), [recommend_movies(1)] * 5, label='Collaborative Filtering')
plt.xlabel('Top N Recommendations')
plt.ylabel('Movies')
plt.legend()
plt.show()

ParserError: Error tokenizing data. C error: EOF inside string starting at row 4682